In [6]:
import pandas as pd
import chess

In [7]:
df_chess = pd.read_csv('..\chess_games.csv')

In [22]:
board = chess.Board()
print(board.fen())
for move in df_chess['moves'][3].split(" "):
    board.push_san(move)
print(board.fen())
print(board)

rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
7r/R5Qp/2p3Nk/3pP1p1/8/4P1P1/1P2BP1P/5RK1 b - - 0 31
. . . . . . . r
R . . . . . Q p
. . p . . . N k
. . . p P . p .
. . . . . . . .
. . . . P . P .
. P . . B P . P
. . . . . R K .


In [28]:
def get_fen_list(moves : str)->list:
    """Transform a string of moves in SAN into a list of FEN board positions for each of the string's moves.

    Args:
        moves (str): string containing the moves in Standart Algebraic Notation separated by an empty space.

    Returns:
        fen_list (list str): list of the Forsty-Edwards Notation of the current board position of each turn.
    """
    # defines a new board in the starting position
    board = chess.Board()
    # creates a list of FEN's
    fen_list = [board.fen()]
    # transform the string in a list with the moves
    moves = moves.split(" ")

    for mv in moves:
        # aplica o movimento ao tabuleiro
        board.push_san(mv)
        fen_list.append(board.fen())
    
    return fen_list
    

In [50]:
def get_fen_dict(fen: str)->dict:
    placement, active_color, castling, en_passant, halfmove_clock, fullmove_clock = fen.split(" ")
    return {
        "placement":     placement,
        "active_color":  active_color,
        "castling":      castling,
        "en_passant":    en_passant,
        "halfmove_clock":halfmove_clock,
        "fullmove_clock": fullmove_clock
    }

In [62]:
def get_turn(active_color:str, fullmove_clock:int)->int:
    """returns the current turn,
    turn 0: initial board position
    odd turns: white moves
    even turns: black moves"""

    # the active color represents the next color to move, so the parity is inverted
    if active_color.lower() == 'w':
        return 2*(fullmove_clock - 1)
    elif active_color.lower() == 'b':
        return 2*fullmove_clock - 1
    else:
        raise ValueError("active_color must be 'w' or 'b'.")


In [38]:
print(get_fen_list(df_chess['moves'][0]))

['rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1', 'rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2', 'rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBNR b KQkq - 0 2', 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3', 'rnbqkbnr/pp2pppp/2p5/3P4/3P4/8/PP2PPPP/RNBQKBNR b KQkq - 0 3', 'rnbqkbnr/pp3ppp/2p1p3/3P4/3P4/8/PP2PPPP/RNBQKBNR w KQkq - 0 4', 'rnbqkbnr/pp3ppp/2p1P3/8/3P4/8/PP2PPPP/RNBQKBNR b KQkq - 0 4', 'rnbqkbnr/pp4pp/2p1p3/8/3P4/8/PP2PPPP/RNBQKBNR w KQkq - 0 5', 'rnbqkbnr/pp4pp/2p1p3/8/3P4/5N2/PP2PPPP/RNBQKB1R b KQkq - 1 5', 'rnbqk1nr/pp4pp/2p1p3/8/1b1P4/5N2/PP2PPPP/RNBQKB1R w KQkq - 2 6', 'rnbqk1nr/pp4pp/2p1p3/8/1b1P4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 3 6', 'rnbqk1nr/pp4pp/2p1p3/b7/3P4/2N2N2/PP2PPPP/R1BQKB1R w KQkq - 4 7', 'rnbqk1nr/pp4pp/2p1p3/b7/3P1B2/2N2N2/PP2PPPP/R2QKB1R b KQkq - 5 7']


In [83]:
def get_piece_df(fen: str)->pd.DataFrame:    
    # get only the placement part of the fen string (e.g. rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR)
    fen_dict = get_fen_dict(fen)
    placements = fen_dict['placement'].split("/")

    piece_list = []
    # iterates over the rows
    for i in range(len(placements)):
        row = placements[i]
        # iterates over the columns
        for j in range(len(row)):
            col = row[j]
            if not col.isdigit():
                turn = None
                piece = col
                if col.islower():
                    color = 'w'
                else:
                    color = 'b'
                x = i + 1
                y = j + 1
                piece_list.append([turn, piece, color, x, y])
    df = pd.DataFrame(piece_list, columns=['turn', 'piece', 'color', 'x', 'y'])
    return df

,turn,piece,color,x,y
0,None,r,w,1,1
1,None,n,w,1,2
2,None,b,w,1,3
3,None,q,w,1,4
4,None,k,w,1,5
5,None,b,w,1,6
6,None,n,w,1,7
7,None,r,w,1,8
8,None,p,w,2,1
9,None,p,w,2,2


(1, 1)
